<link rel="stylesheet" type="text/css" href="custom.css">

# <font color=teal>imports</font>

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../src"))

In [2]:
from src import *


# <font color=teal>housekeeping</font>

In [3]:
db = database_loader.DatabaseLoader(get_config('connection_string'))
DEBUG=True
COMMIT_TO_DATABASE=True
SCHEMA='controls'

# <font color=teal>game info data</font>
Aggregated up from the play-by-play dataset.

We want each team to have its own record for each season and week.

So for any given game there will be two records, one for the home team having its stats, and another for the away team - having the opposite stats

In [ ]:
# We want each team to have a record for each season and week.

game_df = db.query_to_df("""
   --home team labels
    select season,
           week,
           home_team                 as team,
           home_score                as team_score,
           home_coach                as team_coach,
           away_team                 as opposing_team,
           away_score                as opposing_score,
           away_coach                as opposing_coach,
           (home_score - away_score) as spread,
           count(*)

    from controls.game_info G
    group by season, week, home_team, home_score, away_score, away_team, home_coach, away_coach
    UNION ALL
    --away team labels
    select season,
           week,
           away_team                 as team,
           away_score                as team_score,
           away_coach                as team_coach,
           home_team                 as opposing_team,
           home_score                as opposing_score,
           home_coach                as opposing_coach,
           (away_score - home_score) as spread,
           count(*)

    from controls.game_info G
    group by season, week, home_team, home_score, away_score, away_team, home_coach, away_coach
""")

game_df.head()

### <font color="#9370DB">A single game should have two 'opposite' records</font>
let's check that out for the 2017 match between DEN and NYG

In [ ]:
game_df.loc[(game_df.season==2017) & (game_df.week==6) & (game_df.team.isin(['DEN', 'NYG']))]

### <font color="#9370DB">There should be no team with two records for any give week</font>
let's validate that

In [ ]:
# fail if there are any group counts > 1
double_counts = game_df.loc[(game_df['count'].astype(int) > 1)].shape[0]
assert double_counts == 0

# <font color=teal>next gen stats passing<font/>
group by <font color=red>season, week, team</font> ( and top-passing-player_position )

In [ ]:
%%time
ngs_air_power = db.query_to_df("""
    with base as (
    select season, week, team_abbr as team,
           pass_touchdowns,
           avg_time_to_throw,
           avg_completed_air_yards,
           avg_intended_air_yards,
           avg_air_yards_differential,
           aggressiveness,
           max_completed_air_distance,
           avg_air_yards_to_sticks,
           attempts,
           pass_yards,
           interceptions,
           passer_rating,
           completions,
           completion_percentage,
           expected_completion_percentage,
           completion_percentage_above_expectation,
           avg_air_distance,
           max_air_distance,
        row_number() over (partition by season, week, team_abbr, player_position order by pass_yards desc) as rn
    from controls.nextgen_pass
--    where season=2016 and week=1 and team_abbr = 'CHI'
    order by team_abbr, player_position, season desc, week )
    select * from base where rn = 1 and week > 0
""")

ngs_air_power.head()


## <font color=teal>next gen stats rushing<font/>
group by <font color=red>season, week, team</font>

In [ ]:
%%time
ngs_ground_power = db.query_to_df("""
with base as (
    select season, week, team_abbr as team, rush_yards,
           efficiency,
           percent_attempts_gte_eight_defenders,
           avg_time_to_los,
           rush_attempts,
           expected_rush_yards,
           rush_yards_over_expected,
           avg_rush_yards,
           rush_yards_over_expected_per_att,
           rush_pct_over_expected,
           rush_touchdowns,
           player_gsis_id,
           player_first_name,
           player_last_name,
           player_jersey_number,
           player_short_name,
           row_number() over (partition by season, week, team_abbr order by rush_yards desc) as rn
    from controls.nextgen_rush
    order by  team_abbr, season desc, week)
select * from base where week > 0
""" )

ngs_ground_power.head()

# <font color=teal>play-by-play events<font/>
players are called out for certain events like fumbles, touchdowns, etc. in play-by-play
we already picked these out during the transform step,
  and expanded so that each team has its own records irrespective of the opposing team played.
Now we pivot and sum all events by  <font color=red>season, week, team</font>

In [ ]:
%%time
pbp_events = db.query_to_df("""
SELECT
    season, week, team,
    SUM(CASE WHEN event = 'fumble' THEN 1 else 0 END) AS fumble,
    SUM(CASE WHEN event = 'own_kickoff_recovery' THEN 1 else 0 END) AS own_kickoff_recovery,
    SUM(CASE WHEN event = 'safety' THEN 1 else 0 END) AS safety,
    SUM(CASE WHEN event = 'tackle' THEN 1 else 0 END) AS tackle,
    SUM(CASE WHEN event = 'qb_hit' THEN 1 else 0  END) AS qb_hit,
    SUM(CASE WHEN event = 'touchdown' THEN 1  else 0 END) AS touchdown,
    SUM(CASE WHEN event = 'interception' THEN 1 else 0 END) AS interception,
    SUM(CASE WHEN event = 'sack' THEN 1 else 0 END) AS sack
FROM controls.player_events where week > 0
group by season, week, team
order by season desc, team, week
""")

pbp_events.head()

# <font color=teal>player stats<font/>
Each player's stats by are collected by game and play
For this dimension reduction exercise we roll up to <font color=red>season, week, team</font>

In [ ]:
%%time
player_stats = db.query_to_df("""
select
    season,
    week,
    team,
    sum(completions) as ps_completions,
    sum(attempts) as ps_attempts,
    sum(passing_yards) as passing_yards,
    sum(passing_tds) as passing_tds,
    sum(interceptions) as ps_interceptions,
    sum(sacks) as sacks,
    sum(sack_yards) as sack_yards,
    sum(sack_fumbles) as sack_fumbles,
    sum(sack_fumbles_lost) as sack_fumbles_lost,
    sum(passing_air_yards) as passing_air_yards,
    sum(passing_yards_after_catch) as passing_yards_after_catch,
    sum(passing_first_downs) as passing_first_downs,
    avg(passing_epa) as passing_epa,
    sum(passing_2pt_conversions) as passing_2pt_conversions,
    avg(pacr) as avg_pacr,
    avg(dakota) as avg_dakota,
    sum(carries) as carries,
    sum(rushing_yards) as rushing_yards,
    sum(rushing_tds) as rushing_tds,
    sum(rushing_fumbles) as rushing_fumbles,
    sum(rushing_fumbles_lost) as rushing_fumbles_lost,
    sum(rushing_first_downs) as rushing_first_downs,
    avg(rushing_epa) as avg_rushing_epa,
    sum(rushing_2pt_conversions) as rushing_2pt_conversions,
    sum(receptions) as receptions,
    sum(targets) as targets,
    sum(receiving_yards) as receiving_yards,
    sum(receiving_tds) as receiving_tds,
    sum(receiving_fumbles) as receiving_fumbles,
    sum(receiving_fumbles_lost) as receiving_fumbles_lost,
    sum(receiving_air_yards) as receiving_air_yards,
    sum(receiving_yards_after_catch) as receiving_yards_after_catch,
    sum(receiving_first_downs) as receiving_first_downs,
    avg(receiving_epa) as avg_receiving_epa,
    sum(receiving_2pt_conversions) as receiving_2pt_conversions,
    sum(racr) as racr,
    sum(target_share) as target_share,
    sum(air_yards_share) as air_yards_share,
    sum(wopr) as wopr,
    sum(special_teams_tds) as special_teams_tds
from controls.player_stats
group by season,
week,
team
order by season desc, team,  week
""")

player_stats.head()

# <font color=teal>play by play analytics</font>
Analytics are part of the play-by-play dataset - they are collected for each play in each game.

For this dimensionality reduction step we roll them up to the player stats level.

The stats we get are for each play, and those probabilities
  and play level incrementals like WPA and EPA don't make sense in a rollup like this (I think)
  so for this rollup we'll use just EP and WP and well take the first and last metric form each season, week, team groping

We'll also separate home and away teams into their own set, so that for each game there will be two separate sets of analytics, one for the home team and one fr the away team.

### <font color="#9370DB">helper functions</font>

In [ ]:
from typing import List, NamedTuple

class Col(NamedTuple):
    name: str
    alias: str

def build_pivot_sql(team_col: str,  pivot_cols: List[Col], additional_cols: List[Col]):
    db_table = 'controls.play_analytics'

    base_cols = ['season',
                 'week']
    metrics = []
    for col in pivot_cols:
        metrics.append(
            f"""
            MAX(CASE WHEN RN = 1 THEN {col.alias} END) AS start_{col.alias},
            MAX(CASE WHEN RN = (total_rows/2) THEN {col.alias} END) AS half_{col.alias}""")

    inner_cols = base_cols +  [f"{team_col} as team "] + [f"{p.name} as {p.alias}" for p in pivot_cols] + [f"{p.name} as {p.alias}" for p in additional_cols]
    inner_select = ",".join(inner_cols)

    outer_cols = base_cols + ["team"] +  [f"{p.alias}" for p in additional_cols] + metrics
    outer_select = ",".join(outer_cols)
    outer_group = ",".join(base_cols + ["team"] +  [f"{p.alias}" for p in additional_cols])

    station_cols = base_cols + [team_col]
    stations = ",".join(station_cols)

    sql = f"""
    WITH ranked_rows AS (
        SELECT {inner_select},
             ROW_NUMBER() OVER (PARTITION BY
                {stations} ORDER BY play_counter) AS RN,
             COUNT(*) OVER (PARTITION BY
                {stations}) AS total_rows
        FROM {db_table}
    )
    SELECT
        {outer_select}
    FROM ranked_rows
    GROUP BY {outer_group}
    order by season desc, team, week
    """

    if DEBUG:
        print(sql)
    return sql

def build_pivot(team_col: str,  pivot_cols: List[Col],  additional_cols: List[Col]):
    sql = build_pivot_sql(team_col, pivot_cols, additional_cols)
    df = db.query_to_df( sql )
    return df


### <font color="#9370DB">home team statistics</font>

In [ ]:
%%time
pivot_cols = [
    Col('home_wp', "team_wp"),
    Col('away_wp', "opponent_wp"),
]

home_analytics_df = build_pivot(team_col="home_team", pivot_cols=pivot_cols, additional_cols=[Col("away_team", "opponent")])
home_analytics_df.head()

### <font color="#9370DB">away team statistics</font>

In [ ]:
%%time
pivot_cols = [
    Col('away_wp', "team_wp"),
    Col('home_wp', "opponent_wp")
]

away_analytics_df = build_pivot(team_col="away_team", pivot_cols=pivot_cols, additional_cols=[Col("home_team", "opponent")])
away_analytics_df.head()


### <font color="#9370DB">home and away team statistics appended together</font>

In [ ]:
team_analytics = pd.concat([home_analytics_df, away_analytics_df])
team_analytics.head()

# <font color=teal>validate that all of these datasets will merge 'horizontally'</font>
We want one record for each season, week and team.

The metrics themselves pivot horizontally as columns - all the metrics we created above become columns in this final dataset
so we expect that with each merge the number of columns grows, but the row count stays the same

Since we are using this dataset for dimensionality reduction it's ok if we loose a few rows on the join.


### <font color="#9370DB">helper functions</font>

In [ ]:
from pandas import DataFrame

def calc_coverage(title: str, df: DataFrame):
    first = df.season.min()
    last = df.season.max()
    first_wk = df.week.min()
    last_wk = df.week.max()
    seasons = df.season.nunique()
    print(f"Shape of {title:30}:  {df.shape},\t Contains {seasons} seasons, starting with {first} and ending in {last} min week: {first_wk}, max week : {last_wk}")

def print_columns(title, df):
    print(f"\n---------\n{title.strip()} colums")
    for col in df.columns:
        print(col)

### <font color="#9370DB">get shapes before merge</font>

In [ ]:
calc_coverage("Team analytics ", team_analytics)
calc_coverage("ngs_air_power  ", ngs_air_power)
calc_coverage("ngs_ground_power ", ngs_ground_power)
calc_coverage("pbp_events  ", pbp_events)
calc_coverage("player_stats  ", player_stats)
calc_coverage("game info  ", game_df)

In [ ]:
ngs_air_power.drop(columns=['rn'], inplace=True)
ngs_ground_power.drop(columns=['rn'], inplace=True)

In [ ]:
if DEBUG:
    print_columns("Team analytics ", team_analytics)
    print_columns("ngs_air_power  ", ngs_air_power)
    print_columns("ngs_ground_power ", ngs_ground_power)
    print_columns("pbp_events  ", pbp_events)
    print_columns("player_stats  ", player_stats)
    print_columns("game Info     ", game_df)

### <font color="#9370DB">get shapes after each  merge</font>

In [ ]:
merged = pd.merge(ngs_ground_power, ngs_air_power, on=['season', 'week', 'team'])
print("shape after merging ngs_ground_power + ngs_air_power ", merged.shape)
merged = pd.merge(merged, pbp_events, on=['season', 'week', 'team'])
print("shape after merging merged + pbp_events              ", merged.shape)
merged = pd.merge(merged, player_stats, on=['season', 'week', 'team'])
print("shape after merging merged + player_stats            ", merged.shape)
merged = pd.merge(merged, team_analytics, on=['season', 'week', 'team'])
print("shape after merging merged + team_analytics          ", merged.shape)
merged = pd.merge(merged, game_df, on=['season', 'week', 'team'])
print("shape after merging merged + game_df                 ", merged.shape)

merged.shape

### <font color="#9370DB">verify that there are no team weeks with more than one record</font>

In [ ]:
%%time
overlaps = 0
for col in merged.columns:
    if str(col).endswith("_y") or str(col).endswith("_x") or str(col) == "rn":
        print(col)
        overlaps += 1

assert overlaps == 0


# <font color=teal>review and impute our new dataset</font>

### <font color="#9370DB">review our dataset</font>

In [ ]:
team_week_stats = merged
total_rows = team_week_stats.shape[0]
team_week_stats.shape

In [ ]:
team_week_stats.describe().T

### <font color="#9370DB">impute missing values</font>

In [ ]:
missing_values = team_week_stats.isnull().sum().sort_values(ascending=False)
percentage_missing = (missing_values / total_rows) * 100
percentage_missing = percentage_missing.reset_index()
percentage_missing.columns = ['column', 'percentage_missing']
percentage_missing.head()


In [ ]:
# remove analytics columns that are null -- we can't zero them because that changes the overall scoring - they are all expeceted values anyway so should not really be included
# team_week_stats.drop(columns=['expected_rush_yards',  ])
cols = percentage_missing.loc[(percentage_missing.percentage_missing > 0), 'column'].to_numpy()
team_week_stats.drop(columns=cols, inplace=True)
team_week_stats.isnull().sum().sort_values(ascending=False)

### <font color="#9370DB">interactively review distributions and decide which columns to keep</font>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


def hist_charts(numeric_columns):
    # Calculate the number of rows and columns for the grid
    num_cols = 4
    num_rows = (len(numeric_columns.columns) + num_cols - 1) // num_cols

    # Generate separate histograms using seaborn for each numeric column
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 3*num_rows))
    for i, column in enumerate(numeric_columns.columns):
        row = i // num_cols
        col = i % num_cols
        sns.set(style="ticks")
        sns.histplot(data=numeric_columns[column], bins=30, kde=True, ax=axes[row, col])
        axes[row, col].set_title(f"Histogram of {column}")

    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plots
    plt.show()

In [ ]:
# review one at a time
sns.histplot(team_week_stats['pass_touchdowns'])
plt.show()

In [ ]:
# review all numeric columns

# columns we've already decided to keep or drop
numeric_columns_to_drop={'start_team_wp', 'wopr', 'start_team_wp', 'half_team_wp', 'start_opponent_wp', 'half_opponent_wp',  'racr', 'avg_rushing_epa', 'avg_receiving_epa', 'target_share', 'passing_epa', 'avg_pacr', 'sack', 'count', 'interception', 'air_yards_share'}
numeric_columns_to_hide={'season','week', 'opposing_score', 'team_score', 'spread', 'interceptions'}

# all numeric columns and their values
numeric_columns = team_week_stats.select_dtypes(include='number').drop(columns=list(numeric_columns_to_drop.union(numeric_columns_to_hide)) ) ## review whats left over

# columns with less than '20' unique values are really categories - they don't really have a distribution worth looking at per-se
categorical_columns = set()
for column in numeric_columns.columns:
    n = len(numeric_columns[column].value_counts())
    if n < 20:
        categorical_columns.add(column)

# drop categorical columns from our numeric dataset
numeric_columns = numeric_columns.drop(columns=list(categorical_columns))

# review the distribution of the remaining columns
hist_charts(numeric_columns)

### <font color="#9370DB">drop the columns we don't need</font>

In [ ]:
team_week_stats.drop(columns=list(numeric_columns_to_drop), inplace=True)

### <font color="#9370DB">make sure our potential labels for the upcoming dimension analysis is complete</font>

In [ ]:
assert 0 == team_week_stats.spread.isna().sum()
assert 0 == team_week_stats.team_score.isna().sum()
assert 0 == team_week_stats.opposing_score.isna().sum()

### <font color="#9370DB">store to database</font>

In [ ]:
%%time
if COMMIT_TO_DATABASE:
    db.load_table(merged, table_name="team_weekly_stats", schema=SCHEMA, handle_exists="replace")